In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt

In [2]:
(x_train,y_train),(x_test,y_test) = mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [6]:

x_train = x_train.astype('float32') / 255.0
x_test  = x_test.astype('float32')  / 255.0

In [7]:
x_train = x_train.reshape((len(x_train), 784))
x_test  = x_test.reshape((len(x_test), 784))

In [8]:
train_normal = x_train[y_train == 3]
test_normal = x_test[y_test == 3]

In [9]:
input_img = Input(shape=(784,))
encoded = Dense(128, activation='relu')(input_img)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(32, activation='relu')(encoded)

In [10]:
decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(784, activation='sigmoid')(decoded)

In [11]:
autoencoder = Model(input_img, decoded)

In [12]:
autoencoder.compile(optimizer='adam', loss='mae')
autoencoder.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       100,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 784)            │       101,136 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 222,384 (868.69 KB)

 Trainable params: 222,384 (868.69 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
 autoencoder.fit(train_normal, train_normal,
                epochs=30,
                batch_size=256,
                shuffle=True,
                validation_data=(test_normal,test_normal))

Epoch 1/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - loss: 0.4961 - val_loss: 0.4564
Epoch 2/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.3644 - val_loss: 0.0065
Epoch 3/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0023 - val_loss: 5.6580e-04
Epoch 4/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 5.5857e-04 - val_loss: 5.6335e-04
Epoch 5/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 5.5527e-04 - val_loss: 5.6322e-04
Epoch 6/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 5.5820e-04 - val_loss: 5.6320e-04
Epoch 7/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 5.5631e-04 - val_loss: 5.6318e-04
Epoch 8/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 5.5596e-04 - val_loss: 5.6317e-04
Epoch 9/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 5.5809e-04 - val_loss: 5.6316e-04
Epoch 10/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 5.5417e-04 - val_loss: 5.6315e-04
Epoch 11/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 5.5654e-04 - val_loss: 5.6313

In [14]:
normal_test = x_test[y_test==3]
anomaly_test = x_test[y_test==7]

In [15]:
test_data = np.concatenate([normal_test[:100], anomaly_test[:100]])
test_labels = np.array([0]*100 + [1]*100)


In [16]:
recon = autoencoder.predict(test_data)
errors = np.mean(np.abs(test_data - recon),axis=1)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


In [17]:
# threshold
threshold = np.mean(errors[test_labels==0]) + 2*np.std(errors[test_labels==0])
print("Threshold =", threshold)
print("Anomalies Detected =", np.sum(errors > threshold))
print("Total Samples =", len(test_data))

Threshold = 0.0008500293
Anomalies Detected = 5
Total Samples = 200
